In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from Transformer import Create_Transformer
from tensorflow.keras.preprocessing.image import ImageDataGenerator


2025-05-18 00:00:21.073543: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-18 00:00:21.073969: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-18 00:00:21.076730: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-18 00:00:21.083526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747515621.094273   15193 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747515621.09

# Load Models 

## AlexNet


In [2]:
AlexNet = tf.keras.models.load_model('Models/AlexNet/alexnet_model.keras')

E0000 00:00:1747515623.052530   15193 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1747515623.052885   15193 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Inception Model

In [3]:
Inception_model = tf.keras.models.load_model('Models/inception_model/inception_model.keras')

## Transformer Model

In [4]:
IMAGE_SIZE =224   
CHANNELS = 3
NUM_CLASSES = 7 

transformer_model = Create_Transformer(
    image_size=IMAGE_SIZE,
    num_classes=NUM_CLASSES,
    channels=CHANNELS
)

transformer_model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-4),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)
transformer_model.load_weights('Models/Transformer/Transformer_final.weights.h5')

/home/nok/Downloads/Discussion /env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/nok/Downloads/Discussion /env/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 2 variables whereas the saved optimizer has 228 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## Test Script

In [5]:
Test_path ='testNew'

models = {
    "AlexNet": AlexNet,
    "Inception": Inception_model,
    "Transformer": transformer_model
}

class_mapping = {
    'banana_overripe': 0,
    'banana_ripe': 1,
    'banana_rotten': 2,
    'banana_unripe': 3,
    'tomato_fully_ripened': 4,
    'tomato_green': 5,
    'tomato_half_ripened': 6
}

reverse_class_mapping = {v: k for k, v in class_mapping.items()}


test_data_gen = ImageDataGenerator(    
    rescale=1./255,
    rotation_range=40,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest')

test_image_files = sorted([f for f in os.listdir(Test_path) if os.path.isfile(os.path.join(Test_path, f))])

test_data = test_data_gen.flow_from_dataframe(
    pd.DataFrame({'filename': test_image_files}),
    directory=Test_path,
    x_col='filename',
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

for model_name, model in models.items():
    test_preds = model.predict(test_data)
    predicted_classes = np.argmax(test_preds, axis=1)
    image_ids = test_image_files
    submission_df = pd.DataFrame({'ImageID': image_ids, 'Class': predicted_classes})
    submission_df.to_csv(f'CHP_2 Submissions/submission_of_{model_name}.csv', index=False)
    print(f"Submission file 'submission_of_{model_name}.csv' has been created successfully!")

print("All models have been evaluated and submission files have been created.")

Found 2484 validated image filenames.


/home/nok/Downloads/Discussion /env/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1747515630.114655   15286 service.cc:152] XLA service 0x760a04009040 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747515630.114675   15286 service.cc:160]   StreamExecutor device (0): Host, Default Version
2025-05-18 00:00:30.120363: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747515630.229463   15286 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of t

78/78 ━━━━━━━━━━━━━━━━━━━━ 32s 410ms/step
Submission file 'submission_of_AlexNet.csv' has been created successfully!
78/78 ━━━━━━━━━━━━━━━━━━━━ 171s 2s/step
Submission file 'submission_of_Inception.csv' has been created successfully!
78/78 ━━━━━━━━━━━━━━━━━━━━ 114s 1s/step
Submission file 'submission_of_Transformer.csv' has been created successfully!
All models have been evaluated and submission files have been created.
